In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame as df
import xarray as xr
import netCDF4 as nc
import os
import requests
from datetime import timedelta
import tensorflow as tf
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [4]:
## Read all net files
atomFeatDF = pd.DataFrame()
dataDirPath = '/Volumes/JACk_HD/omegaslpwind/'
fileList = os.listdir(dataDirPath)
for i in range(len(fileList)):
    if(fileList[i].startswith("19")|fileList[i].startswith("20")) == False:
        continue
    tempDF = pd.DataFrame()
    time = datetime.strptime(fileList[i].split('.')[6], '%Y%m')
    fileData = xr.open_dataset(dataDirPath+fileList[i])
    tempDF = fileData.to_dataframe()
    tempDF = tempDF.reset_index()
    tempDF['time'] = time
    atomFeatDF = pd.concat([atomFeatDF,tempDF])
    
salt = pd.DataFrame()
dataDirPath = '/Volumes/JACk_HD/sssmass/'
fileList = os.listdir(dataDirPath)
for i in range(len(fileList)):
    tempDF = pd.DataFrame()
    time = datetime.strptime(fileList[i].split('.')[6], '%Y%m')
    fileData = xr.open_dataset(dataDirPath+fileList[i])
    tempDF = fileData.to_dataframe()
    tempDF = tempDF.reset_index()
    tempDF['time'] = time
    salt = pd.concat([salt,tempDF])
    
finalFeatDF = atomFeatDF.merge(salt,how = 'inner',on = ['time','lon','lat'])
finalFeatDF['month'] = finalFeatDF['time'].dt.month
finalFeatDF['year'] = finalFeatDF['time'].dt.year
finalFeatDF

,time,lon,lat,OMEGA500,SLP,U500,SSSMASS,month,year
0,1980-02-01,-100.0,10.0,0.016336,101199.953125,-2.551559,6.593476e-08,2,1980
1,1980-02-01,-100.0,10.5,0.015123,101212.867188,-2.469250,7.231588e-08,2,1980
2,1980-02-01,-100.0,11.0,0.012895,101223.859375,-2.331853,7.757134e-08,2,1980
3,1980-02-01,-100.0,11.5,0.010122,101233.234375,-2.125592,8.017119e-08,2,1980
4,1980-02-01,-100.0,12.0,0.007562,101242.484375,-1.876976,8.013701e-08,2,1980
...,...,...,...,...,...,...,...,...,...
4021054,2022-08-01,-20.0,38.0,0.038059,102121.734375,3.044182,1.971932e-08,8,2022
4021055,2022-08-01,-20.0,38.5,0.037125,102129.007812,2.946491,1.927289e-08,8,2022
4021056,2022-08-01,-20.0,39.0,0.036537,102135.195312,2.932284,1.907663e-08,8,2022
4021057,2022-08-01,-20.0,39.5,0.035774,102139.859375,2.949570,1.901475e-08,8,2022


In [129]:
sstFeatDF = pd.DataFrame()
dataDirPath = 'ersstv5/'
fileList = os.listdir(dataDirPath)
fileList.remove( '.DS_Store')

In [132]:
fileListTime = [int(i.split('.')[2]) for i in fileList]

In [141]:
sortedList = [fileList[i] for i in np.argsort(fileListTime)]
sortedList[-5:]

['ersst.v5.202108.nc',
 'ersst.v5.202109.nc',
 'ersst.v5.202110.nc',
 'ersst.v5.202111.nc',
 'ersst.v5.202112.nc']

In [142]:
for i in range(len(sortedList)):
    if (sortedList[i].endswith('.nc') == False):
        continue
    tempDF = pd.DataFrame()
    time = datetime.strptime(sortedList[i].split('.')[2], '%Y%m')
    fileData = xr.open_dataset(dataDirPath+sortedList[i])
    tempDF = fileData.to_dataframe()
    tempDF = tempDF.reset_index()
    tempDF['time'] = time
    sstFeatDF = pd.concat([sstFeatDF,tempDF])

In [154]:
sstFeatDF = sstFeatDF.loc[(sstFeatDF['lon'] >= 260) & (sstFeatDF['lon'] <= 340) & (sstFeatDF['lat'] >= 10) & (sstFeatDF['lat'] <= 40) & (sstFeatDF['time'].dt.month <= 5)]

In [157]:
sstFeatDF = sstFeatDF.drop(columns='lev',axis = 1)

In [180]:
sstFeatDF['year'] = sstFeatDF['time'].dt.year
sstFeatDF['month'] = sstFeatDF['time'].dt.month

In [182]:
sstFeatDF.to_csv("../Data/ERSSTv5/ERSSTv5_NotInterpolated.csv")

In [163]:
finalFeatDF = pd.read_csv('../Data/MERRA2/Features.csv')

In [165]:
finalFeatDF = finalFeatDF.drop('Unnamed: 0',axis = 1)

In [172]:
np.unique(finalFeatDF['lat'])

array([10. , 10.5, 11. , 11.5, 12. , 12.5, 13. , 13.5, 14. , 14.5, 15. ,
       15.5, 16. , 16.5, 17. , 17.5, 18. , 18.5, 19. , 19.5, 20. , 20.5,
       21. , 21.5, 22. , 22.5, 23. , 23.5, 24. , 24.5, 25. , 25.5, 26. ,
       26.5, 27. , 27.5, 28. , 28.5, 29. , 29.5, 30. , 30.5, 31. , 31.5,
       32. , 32.5, 33. , 33.5, 34. , 34.5, 35. , 35.5, 36. , 36.5, 37. ,
       37.5, 38. , 38.5, 39. , 39.5, 40. ])

In [174]:
np.unique(sstFeatDF['lat'])

array([10., 12., 14., 16., 18., 20., 22., 24., 26., 28., 30., 32., 34.,
       36., 38., 40.])

In [176]:
np.unique(sstFeatDF['lon'])-360

array([-100.,  -98.,  -96.,  -94.,  -92.,  -90.,  -88.,  -86.,  -84.,
        -82.,  -80.,  -78.,  -76.,  -74.,  -72.,  -70.,  -68.,  -66.,
        -64.,  -62.,  -60.,  -58.,  -56.,  -54.,  -52.,  -50.,  -48.,
        -46.,  -44.,  -42.,  -40.,  -38.,  -36.,  -34.,  -32.,  -30.,
        -28.,  -26.,  -24.,  -22.,  -20.])

In [183]:
np.unique(finalFeatDF['lon'])

array([-100.   ,  -99.375,  -98.75 ,  -98.125,  -97.5  ,  -96.875,
        -96.25 ,  -95.625,  -95.   ,  -94.375,  -93.75 ,  -93.125,
        -92.5  ,  -91.875,  -91.25 ,  -90.625,  -90.   ,  -89.375,
        -88.75 ,  -88.125,  -87.5  ,  -86.875,  -86.25 ,  -85.625,
        -85.   ,  -84.375,  -83.75 ,  -83.125,  -82.5  ,  -81.875,
        -81.25 ,  -80.625,  -80.   ,  -79.375,  -78.75 ,  -78.125,
        -77.5  ,  -76.875,  -76.25 ,  -75.625,  -75.   ,  -74.375,
        -73.75 ,  -73.125,  -72.5  ,  -71.875,  -71.25 ,  -70.625,
        -70.   ,  -69.375,  -68.75 ,  -68.125,  -67.5  ,  -66.875,
        -66.25 ,  -65.625,  -65.   ,  -64.375,  -63.75 ,  -63.125,
        -62.5  ,  -61.875,  -61.25 ,  -60.625,  -60.   ,  -59.375,
        -58.75 ,  -58.125,  -57.5  ,  -56.875,  -56.25 ,  -55.625,
        -55.   ,  -54.375,  -53.75 ,  -53.125,  -52.5  ,  -51.875,
        -51.25 ,  -50.625,  -50.   ,  -49.375,  -48.75 ,  -48.125,
        -47.5  ,  -46.875,  -46.25 ,  -45.625,  -45.   ,  -44.

### Normalization

In [187]:
normalizedFeat = finalFeatDF.copy()
normalizedFeat[['OMEGA500','SLP','U500','SSSMASS']] = (finalFeatDF[['OMEGA500','SLP','U500','SSSMASS']]-finalFeatDF[['OMEGA500','SLP','U500','SSSMASS']].min())/(finalFeatDF[['OMEGA500','SLP','U500','SSSMASS']].max()-finalFeatDF[['OMEGA500','SLP','U500','SSSMASS']].min())

In [192]:
normalizedFeat.to_csv("../Data/MERRA2/NormalizedFeatures.csv")

In [204]:
sst = pd.read_csv("../Data/ERSSTv5/ERSSTv5_NotInterpolated.csv").drop('Unnamed: 0',axis = 1)
tc = pd.read_csv("../Data/TC Count/IBTrACS/IBTrACS(1980-P, 100W-20W&10N-40N).csv")
tc = tc.loc[tc['SEASON'] < 2022]

In [203]:
sst_mena = sst.groupby('year')

,SEASON,COUNT
0,1980,18
1,1981,22
2,1982,8
3,1983,6
4,1984,20
5,1985,14
6,1986,10
7,1987,14
8,1988,19
9,1989,15


In [205]:
fileData = xr.open_dataset("oisst-avhrr-v02r01.19810901.nc")

In [208]:
sampledf = fileData.to_dataframe().reset_index()

In [209]:
sampledf

,time,zlev,lat,lon,anom,err,ice,sst
0,1981-09-01 12:00:00,0.0,-89.875,0.125,NaN,NaN,NaN,NaN
1,1981-09-01 12:00:00,0.0,-89.875,0.375,NaN,NaN,NaN,NaN
2,1981-09-01 12:00:00,0.0,-89.875,0.625,NaN,NaN,NaN,NaN
3,1981-09-01 12:00:00,0.0,-89.875,0.875,NaN,NaN,NaN,NaN
4,1981-09-01 12:00:00,0.0,-89.875,1.125,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1036795,1981-09-01 12:00:00,0.0,89.875,358.875,0.7,0.3,0.83,-1.1
1036796,1981-09-01 12:00:00,0.0,89.875,359.125,0.7,0.3,0.83,-1.1
1036797,1981-09-01 12:00:00,0.0,89.875,359.375,0.7,0.3,0.83,-1.1
1036798,1981-09-01 12:00:00,0.0,89.875,359.625,0.7,0.3,0.83,-1.1


In [210]:
dataDirPath = '/Volumes/JACk_HD/omegaslpwind/'
fileList = os.listdir(dataDirPath)

In [214]:
fileList[1]

'HTTP_services.cgi?FILENAME=%2Fdata%2FMERRA2_MONTHLY%2FM2TMNXSLV.5.12.4%2F1980%2FMERRA2_100.tavgM_2d_slv_Nx.198002.nc4&FORMAT=bmM0Lw&BBOX=10,-100,40,-20&LABEL=MERRA2_100.tavgM_2d_slv_Nx.198002.SUB.nc&SHORTNAME=M2TMNXSLV&SERVICE=L34RS_MER'

'198001'

In [219]:
datetime.strptime(fileList[0].split('.')[6], '%Y%m')

datetime.datetime(1980, 1, 1, 0, 0)